<a href="https://colab.research.google.com/github/4060E009/AI_and_security/blob/master/10_3%E5%88%86%E5%89%B2%E6%87%89%E7%94%A8%E5%90%88%E4%BD%B52020_11_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://nbviewer.jupyter.org/github/LearnXu/pydata-notebook/blob/master/Chapter-10/10.3%20Apply%EF%BC%9AGeneral%20split-apply-combine%EF%BC%88%E5%BA%94%E7%94%A8%EF%BC%9A%E9%80%9A%E5%B8%B8%E7%9A%84%E5%88%86%E5%89%B2-%E5%BA%94%E7%94%A8-%E5%90%88%E5%B9%B6%EF%BC%89.ipynb

# **General split-apply-combine**

**general-purpose: 可以理解為通用，泛用。**

例子：在計算機軟件中，通用編程語言(General-purpose programming language )**指被設計為各種應用領域服務的編程語言。通常通用編程語言不含有為特定應用領域設計的結構。**

**相對而言，特定域編程語言就是為某一個特定的領域或應用軟件設計的編程語言。比如說，LaTeX就是專門為排版文獻而設計的語言。**

最通用的GroupBy(分組)**方法是apply，這也是本節的主題。如下圖所示，apply會把對象分為多個部分，然後將函數應用到每一個部分上，然後把所有的部分都合並起來**



**返回之前提到的tipping數據集，假設我們想要根據不同組（group），選擇前5個tip_pct值最大的。首先，寫一個函數，函數的功能為在特定的列，選出有最大值的行**

In [3]:
import numpy as np
import pandas as pd

In [4]:
!wget https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv

--2020-11-07 11:47:52--  https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6545 (6.4K) [text/plain]
Saving to: ‘tips.csv’

tips.csv            100%[===================>]   6.39K  --.-KB/s    in 0s      

2020-11-07 11:47:53 (67.5 MB/s) - ‘tips.csv’ saved [6545/6545]



In [5]:
tips = pd.read_csv('tips.csv')

In [6]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [7]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [8]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [9]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


**現在，如果我們按smoker分組，然後用apply來使用這個函數，我們能得到下面的結果**

In [10]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

**如果傳遞一個函數給apply，可以在函數之後，設定其他一些參數**

In [11]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

**試一試在GroupBy對象上調用describe**

In [12]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [13]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [14]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

**在GroupBy內部，當我們想要調用一個像describe這樣的函數的時候，其實相當於下面的寫法：**

**f = lambda x: x.describe()**

**grouped.apply(f)**


# **Suppressing the Group Keys**

**我們可以在groupby函數中設置group_keys=False來關閉這個功能**

In [15]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


# **Quantile and Bucket Analysis**

**把函數通過groupby整合起來，可以在做桶分析或分位數分析的時候更方便。假設一個簡單的隨機數據集和一個等長的桶類型（bucket categorization），使用cut**

In [16]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                       'data2': np.random.randn(1000)})
frame.head()

,data1,data2
0,-0.431798,-1.131884
1,-0.173481,-0.609419
2,0.792998,-0.190936
3,-1.727124,-0.736443
4,-1.432412,1.117179


In [17]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0     (-1.361, 0.184]
1     (-1.361, 0.184]
2      (0.184, 1.728]
3    (-2.912, -1.361]
4    (-2.912, -1.361]
5     (-1.361, 0.184]
6     (-1.361, 0.184]
7     (-1.361, 0.184]
8      (1.728, 3.273]
9     (-1.361, 0.184]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.912, -1.361] < (-1.361, 0.184] < (0.184, 1.728] <
                                    (1.728, 3.273]]

**cut返回的Categorical object（類別對象）能直接傳入groupby。所以我們可以在data2列上計算很多統計值**

In [18]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}


In [19]:
grouped = frame.data2.groupby(quartiles)

In [20]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.912, -1.361]",-2.758308,2.100513,75.0,-0.066076
"(-1.361, 0.184]",-3.163293,2.955798,496.0,0.021410
"(0.184, 1.728]",-3.888441,3.477124,387.0,0.022745
"(1.728, 3.273]",-2.017046,2.059969,42.0,0.302372


**也有相同長度的桶（equal-length buckets）；想要按照樣本的分位數得到相同長度的桶，用qcut。這里設定labels=False來得到分位數的數量**

In [21]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

In [22]:
grouped = frame.data2.groupby(grouping)

In [23]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.758308,2.100513,100.0,-0.028172
1,-2.502089,2.259558,100.0,-0.028424
2,-2.369822,2.292001,100.0,-0.019037
3,-3.163293,2.214728,100.0,0.004149
4,-2.091060,2.485396,100.0,0.140502
5,-2.392882,2.955798,100.0,-0.032189
6,-3.888441,3.360453,100.0,0.099498
7,-2.646949,3.477124,100.0,0.141860
8,-2.447992,1.958589,100.0,-0.045918


# **Example: Filling Missing Values with Group-Specific Values**

**在處理缺失值的時候，一些情況下我們會直接用dropna來把缺失值刪除，但另一些情況下，我們希望用一些固定的值來代替缺失值，而fillna就是用來做這個的，例如，這里我們用平均值mean來代替缺失值NA**

In [24]:
s = pd.Series(np.random.randn(6))

In [25]:
s[::2] = np.nan

In [26]:
s

0         NaN
1    1.613352
2         NaN
3    0.123932
4         NaN
5    1.653668
dtype: float64

In [27]:
s.fillna(s.mean())

0    1.130317
1    1.613352
2    1.130317
3    0.123932
4    1.130317
5    1.653668
dtype: float64

**假設我們想要給每一組填充不同的值。一個方法就是對數據分組後，用apply來調用fillna，在每一個組上執行一次。這里有一些樣本是把美國各州分為西部和東部**

In [28]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

In [29]:
group_key = ['East'] * 4 + ['West'] * 4
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [30]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          0.689508
New York     -1.377439
Vermont      -1.646232
Florida       0.091466
Oregon       -0.423836
Nevada        1.126556
California   -0.284841
Idaho         0.955843
dtype: float64

**我們令data中某些值為缺失值**

In [31]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.689508
New York     -1.377439
Vermont            NaN
Florida       0.091466
Oregon       -0.423836
Nevada             NaN
California   -0.284841
Idaho              NaN
dtype: float64

In [32]:
data.groupby(group_key).mean()

East   -0.198821
West   -0.354338
dtype: float64

**然后我们可以用每个组的平均值来填充NA**

In [33]:
fill_mean = lambda g: g.fillna(g.mean())

In [34]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.689508
New York     -1.377439
Vermont      -0.198821
Florida       0.091466
Oregon       -0.423836
Nevada       -0.354338
California   -0.284841
Idaho        -0.354338
dtype: float64

**在另外一些情況下，我們可能希望提前設定好用於不同組的填充值。因為group有一個name屬性，我們可以利用這個**

In [35]:
fill_values = {'East': 0.5, 'West': -1}

In [36]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [37]:
data.groupby(group_key).apply(fill_func)

Ohio          0.689508
New York     -1.377439
Vermont       0.500000
Florida       0.091466
Oregon       -0.423836
Nevada       -1.000000
California   -0.284841
Idaho        -1.000000
dtype: float64

# **Example: Random Sampling and Permutation**

**假設我們想要從一個很大的數據集里隨機抽出一些樣本，這里我們可以在Series上用sample方法。為了演示，這里縣創建一副模擬的撲克牌**

In [38]:
# Hearts红桃，Spades黑桃，Clubs梅花，Diamonds方片
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

**這樣我們就得到了一個長度為52的Series，索引（index）部分是牌的名字，對應的值為牌的點數，這里的點數是按Blackjack（二十一點）的遊戲規則來設定的。**

**Blackjack（二十一點）: 2點至10點的牌以牌面的點數計算，J、Q、K 每張為10點，A可記為1點或為11點。這里為了方便，我們只把A記為1點。 **

In [39]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

**現在，就像我們上面說的，隨機從牌組中抽出5張牌**

In [40]:
def draw(deck, n=5):
    return deck.sample(n)

In [41]:
draw(deck)

2H     2
KS    10
KH    10
3S     3
AH     1
dtype: int64

**假設我們想要從每副花色中隨機抽取兩張，花色是每張牌名字的最後一個字符（即H, S, C, D），我們可以根據花色分組，然後使用apply**

In [42]:
get_suit = lambda card: card[-1] # last letter is suit

In [43]:
deck.groupby(get_suit).apply(draw, n=2)

C  4C      4
   8C      8
D  AD      1
   10D    10
H  JH     10
   6H      6
S  QS     10
   10S    10
dtype: int64

**另外一種寫法**

In [44]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

2C      2
6C      6
2D      2
4D      4
KH     10
10H    10
10S    10
5S      5
dtype: int64

# **Example: Group Weighted Average and Correlation**

**在groupby的split-apply-combine機制下，DataFrame的兩列或兩個Series，計算組加權平均（Group Weighted Average）是可能的。這里舉個例子，下面的數據集包含組鍵，值，以及權重**

In [45]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-1.228581,0.860648
1,a,1.883813,0.679022
2,a,-2.325747,0.261413
3,a,-0.087960,0.085764
4,b,-0.532899,0.070801
5,b,1.091300,0.646958
6,b,-0.174627,0.052936
7,b,0.981362,0.438628


**按category分組來計算組加權平均**

In [46]:
grouped = df.groupby('category')

In [47]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [48]:
grouped.apply(get_wavg)

category
a   -0.208682
b    0.900920
dtype: float64

**另一個例子，考慮一個從Yahoo！財經上得到的經濟數據集，包含一些股票交易日結束時的股價，以及S&P 500指數(即SPX符號)：**

**標準普爾500指數英文簡寫為S&P 500 Index，是記錄美國500家上市公司的一個股票指數。這個股票指數由標準普爾公司創建並維護。**

**標準普爾500指數覆蓋的所有公司，都是在美國主要交易所，如紐約證券交易所、Nasdaq交易的上市公司。與道瓊斯指數相比，標準普爾500指數包含的公司更多，因此風險更為分散，能夠反映更廣泛的市場變化。** 

In [50]:
!wget https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/stock_px_2.csv

--2020-11-07 12:01:13--  https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/stock_px_2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101402 (99K) [text/plain]
Saving to: ‘stock_px_2.csv’

stock_px_2.csv      100%[===================>]  99.03K  --.-KB/s    in 0.04s   

2020-11-07 12:01:14 (2.57 MB/s) - ‘stock_px_2.csv’ saved [101402/101402]



In [59]:
close_px = pd.read_csv('stock_px_2.csv', parse_dates=True,
                       index_col=0)

In [60]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [61]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


**一個比較有意思的嘗試是計算一個DataFrame，包括與SPX這一列逐年日收益的相關性（計算百分比變化）。一個可能的方法是，我們先創建一個能計算不同列相關性的函數，然後拿每一列與SPX這一列求相關性**

In [62]:
spx_corr = lambda x: x.corrwith(x['SPX'])

**然後我們通過pct_change在close_px上計算百分比的變化**

In [63]:
rets = close_px.pct_change().dropna()

**最後，我們按年來給這些百分比變化分組，年份可以從每行的標簽中通過一個一行函數提取，然後返回的結果中，用datetime標簽來表示年份**

In [64]:
get_year = lambda x: x.year

In [65]:
by_year = rets.groupby(get_year)

In [66]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


**我們也可以計算列內的相關性。這里我們計算蘋果和微軟每年的相關性**

In [67]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

# ** Example: Group-Wise Linear Regression**

**就像上面介紹的例子，使用groupby可以用於更覆雜的組對組統計分析，只要函數能返回一個pandas對象或標量。例如，我們可以定義regress函數（利用statsmodels庫），在每一個數據塊（each chunk of data）上進行普通最小平方回歸（ordinary least squares (OLS) regression）計算**

In [68]:
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [69]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

**現在，按年用蘋果AAPL在標普SPX上做線性回歸**

In [70]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514
